In [ ]:
import pandas as pd
import os

# Path to your CSV files directory
csv_directory = "/content/"

# Get a list of all CSV files in the directory
csv_files = [f for f in os.listdir(csv_directory) if f.endswith('.csv')]

# Function to extract maturity and strike from filename
def extract_info(filename):
    base_name = os.path.splitext(os.path.basename(filename))[0]
    parts = base_name.split('Tmt')[1].split('Str')
    maturity = float(parts[0])
    strike = float(parts[1])
    return maturity, strike

# Function to process CSV file
def process_csv(file_path):
    # Read CSV file
    df = pd.read_csv(file_path)
    # Extract maturity and strike from filename
    maturity, strike = extract_info(file_path)
    # Add maturity and strike as columns
    df['Maturity'] = maturity
    df['Strike'] = strike
    return df

# Process each CSV file and concatenate
all_dataframes = []
for file_name in csv_files:
    file_path = os.path.join(csv_directory, file_name)
    df = process_csv(file_path)
    all_dataframes.append(df)

# Concatenate all datasets
concatenated_df = pd.concat(all_dataframes, ignore_index=True)

# Drop rows containing 'inf' values in any column
df = concatenated_df[~concatenated_df.isin([' inf']).any(axis=1)].reset_index(drop=True)
df = concatenated_df
df = df.rename(columns=lambda x: x.strip())
df = df.astype('float32')
df = df[df['price']<100]
# Print the DataFrame after dropping rows
df

<ipython-input-1-b1d182f1b24e>:21: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
<ipython-input-1-b1d182f1b24e>:21: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
<ipython-input-1-b1d182f1b24e>:21: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
<ipython-input-1-b1d182f1b24e>:21: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
<ipython-input-1-b1d182f1b24e>:21: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
<ipython-input-1-b1d182f1b24e>:21: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
<ipython-i

,alpha,beta,m,rho,Y0,price,95cI,Maturity,Strike
0,0.2,0.379473,0.34,0.95,0.40,0.177166,2.334980e+01,2.0,0.87
1,0.2,0.379473,0.34,0.95,0.35,0.203504,2.704929e+01,2.0,0.87
2,0.2,0.379473,0.34,0.95,0.31,0.225150,2.958679e+01,2.0,0.87
3,0.2,0.379473,0.34,0.95,0.27,0.247051,3.170565e+01,2.0,0.87
4,0.2,0.379473,0.34,0.95,0.23,0.269066,3.342225e+01,2.0,0.87
...,...,...,...,...,...,...,...,...,...
6399875,128.0,22.400000,0.06,-0.55,0.20,78.875664,3.490727e+07,1.0,1.05
6399876,128.0,22.400000,0.06,-0.55,0.17,78.726395,3.473112e+07,1.0,1.05
6399877,128.0,22.400000,0.06,-0.55,0.14,78.576637,3.455773e+07,1.0,1.05
6399878,128.0,22.400000,0.06,-0.55,0.11,78.426895,3.438790e+07,1.0,1.05


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.optim.lr_scheduler import ReduceLROnPlateau
import numpy as np

# Prepare the data
X = df[['alpha', 'beta', 'm', 'rho', 'Y0', 'Maturity', 'Strike']].values
y = df['price'].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define RMSE loss function
def rmse_loss(y_true, y_pred):
    return torch.sqrt(torch.mean((y_true - y_pred)**2))

# Define batch size
batch_size = 32768

# Standardize the data
scaler_X = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

scaler_y = StandardScaler()
y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1, 1)).flatten()  # Scale the target variable
y_test_scaled = scaler_y.transform(y_test.reshape(-1, 1)).flatten()  # Scale the target variable

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)

y_train_tensor = torch.tensor(y_train_scaled, dtype=torch.float32).view(-1, 1)
y_test_tensor = torch.tensor(y_test_scaled, dtype=torch.float32).view(-1, 1)

# Create DataLoader for training and testing sets
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Define the neural network architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(7, 256)  # Increase neurons
        self.bn1 = nn.BatchNorm1d(256)  # BatchNorm layer
        self.fc2 = nn.Linear(256, 512)  # Additional layer
        self.bn2 = nn.BatchNorm1d(512)  # BatchNorm layer
        self.fc3 = nn.Linear(512, 1024)   # Additional layer
        self.bn3 = nn.BatchNorm1d(1024)  # BatchNorm layer
        self.fc4 = nn.Linear(1024, 2048)   # Additional layer
        self.bn4 = nn.BatchNorm1d(2048)  # BatchNorm layer
        self.fc5 = nn.Linear(2048, 1024)   # Additional layer
        self.bn5 = nn.BatchNorm1d(1024)  # BatchNorm layer
        self.fc6 = nn.Linear(1024, 512)   # Additional layer
        self.bn6 = nn.BatchNorm1d(512)  # BatchNorm layer
        self.fc7 = nn.Linear(512, 256)   # Additional layer
        self.bn7 = nn.BatchNorm1d(256)  # BatchNorm layer
        self.fc8 = nn.Linear(256, 1)
        self.relu = nn.ReLU()
        self.dropout1 = nn.Dropout(0.1)
        self.dropout2 = nn.Dropout(0.4)
        self.dropout3 = nn.Dropout(0.6)

    def forward(self, x):
        x = self.relu(self.bn1(self.fc1(x)))
        x = self.dropout1(self.relu(self.bn2(self.fc2(x))))
        x = self.dropout2(self.relu(self.bn3(self.fc3(x))))
        x = self.dropout3(self.relu(self.bn4(self.fc4(x))))
        x = self.dropout3(self.relu(self.bn5(self.fc5(x))))
        x = self.dropout2(self.relu(self.bn6(self.fc6(x))))
        x = self.dropout1(self.relu(self.bn7(self.fc7(x))))
        x = self.fc8(x)
        return x

# Instantiate the model and move it to GPU
model = Net().cuda()

# Define the loss function and optimizer
criterion = rmse_loss
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Define the learning rate scheduler
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=8, factor=0.1, verbose=True)

# Define early stopping parameters
patience = 20
best_loss = float('inf')
counter = 0

# Warm-up phase
warmup_epochs = 2
for epoch in range(warmup_epochs):
    model.train()
    for inputs, labels in train_loader:
        inputs, labels = inputs.cuda(), labels.cuda()
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

# Train the neural network
num_epochs = 150
for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.cuda(), labels.cuda()
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / len(train_dataset)

    # Validation
    model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        running_val_loss = 0.0
        for inputs, labels in test_loader:
            inputs, labels = inputs.cuda(), labels.cuda()
            outputs = model(inputs)
            val_loss = criterion(outputs, labels)
            running_val_loss += val_loss.item() * inputs.size(0)
        epoch_val_loss = running_val_loss / len(test_dataset)

    scheduler.step(epoch_val_loss)

    if epoch_val_loss < best_loss:
        best_loss = epoch_val_loss
        counter = 0
        # Save the best model
        torch.save(model.state_dict(), 'best_model.pth')
    else:
        counter += 1
        if counter >= patience:
            print("Early stopping triggered.")
            break

    if (epoch+1) % 1 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {epoch_loss:.4f}, Val Loss: {epoch_val_loss:.4f}')

# Load the best model
model.load_state_dict(torch.load('best_model.pth'))

# Evaluate the best model on the test set
model.eval()
with torch.no_grad():
    running_loss = 0.0
    for inputs, labels in test_loader:
        inputs, labels = inputs.cuda(), labels.cuda()
        outputs = model(inputs)
        # Unscale predictions
        unscaled_outputs = scaler_y.inverse_transform(outputs.cpu().numpy()).flatten()
        unscaled_labels = scaler_y.inverse_transform(labels.cpu().numpy()).flatten()
        # Calculate RMSE loss on unscaled predictions and labels
        unscaled_loss = criterion(torch.tensor(unscaled_outputs), torch.tensor(unscaled_labels))
        running_loss += unscaled_loss.item() * inputs.size(0)
    unscaled_test_loss = running_loss / len(test_dataset)
    print(f'Unscaled Test Loss: {unscaled_test_loss:.4f}')

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Epoch [1/150], Train Loss: 0.9023, Val Loss: 0.8932
Epoch [2/150], Train Loss: 0.8987, Val Loss: 0.8888
Epoch [3/150], Train Loss: 0.8938, Val Loss: 0.8869
Epoch [4/150], Train Loss: 0.8900, Val Loss: 0.8802
Epoch [5/150], Train Loss: 0.8856, Val Loss: 0.8724
Epoch [6/150], Train Loss: 0.8803, Val Loss: 0.8749
Epoch [7/150], Train Loss: 0.8756, Val Loss: 0.8617
Epoch [8/150], Train Loss: 0.8689, Val Loss: 0.8546
Epoch [9/150], Train Loss: 0.8646, Val Loss: 0.8412
Epoch [10/150], Train Loss: 0.8580, Val Loss: 0.8411
Epoch [11/150], Train Loss: 0.8531, Val Loss: 0.8350
Epoch [12/150], Train Loss: 0.8473, Val Loss: 0.8292
Epoch [13/150], Train Loss: 0.8410, Val Loss: 0.8208
Epoch [14/150], Train Loss: 0.8340, Val Loss: 0.8112
Epoch [15/150], Train Loss: 0.8277, Val Loss: 0.7997
Epoch [16/150], Train Loss: 0.8230, Val Loss: 0.7985
Epoch [17/150], Train Loss: 0.8208, Val Loss: 0.8062
Epoch [18/150], Train Loss: 0.8139, Val Loss: 0.7866
Epoch [19/150], Train Loss: 0.8081, Val Loss: 0.7902
Ep

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(range(len(unscaled_outputs)),unscaled_outputs,label='Predictions')
plt.scatter(range(len(unscaled_labels)),unscaled_labels,label='Truth')
plt.legend()